# Pandas

Version 0.2 Sep 2020

_Pandas_ is another package that, like _NumPy_, is widely used in the scientific community. We'll look at how it is commonly used to read in files into a special data structure called a `DataFrame`. A _Pandas_ `DataFrame` is something like a cross between a 2D _NumPy_ array and a spreadsheet sheet
> The 1D equivalent of a _Pandas_ `DataFrame` is called a `Series` and behaves a bit like a supercharged _NumPy_ array.

_Pandas_ `DataFrame`s allow the data they contain to be processed and manipulated in many ways and we'll look at some of the basic methods that you can use in this notebook.

We've provided a small CSV file called `observatories.csv` which should be put into the same folder as this notebook. If this file were opened in a spreadsheet it would look like this:

![observatories.png](observatories.png)

Notice that the first two rows are either a comment or blank. The third row shows the data column names and the actual data start on the fourth row.

Let's use _Pandas_ to read this file and see how to perform some basic functions.

As you work through this notebook remember to Run each cell in turn.  Some of the later cells depend on statements in earlier cells (for instance the `import` statements at the start are needed for later cells).  If you get an error message in a particular cell, go back and make sure that you have executed all of the statements in the cells above it.

First, import any required packages and modules.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Reading in data

Now let's read in the file to a _Pandas_ `DataFrame`.

In [ ]:
df = pd.read_csv('observatories.csv', sep=',', header=2)

Things to note:

* We've told _Pandas_ that the columns are delimted by `','` - actually this is the default but you can change this to any other character that is used in the file. For example, if the file is tab-delimited use `sep='\t'` or, for space-delimited files, use `sep='\s+'`)

* We've also told _Pandas_ where the data starts, missing out some information lines, starting with the row containing column headings, by using `'header=2'`.

At the end of this notebook there is a section which describes how you can get more control over which data you read in and which you leave out.

So what has this command created? Well, just typing the `DataFrame`'s name produces a nicely formatted representation of its contents. 

In [ ]:
df

Things to note:

* _Pandas_ has recognised that the first actual line it comes to (after the header lines have been skipped) is the actual names of the columns. 

* It has automatically indexed the rows (the numbers on the left) based on the order the rows appear in the file We'll be changing this later, indexing on a column.

### EXERCISE 1.1

Just using `df` gives a nicely formatted output. Try running `print(df)` as an alternative.


In [ ]:
print (df)
# Write your code here...


You can also just look at the first few rows...


In [ ]:
df.head(4)

... or the last few rows.

In [ ]:
df.tail(3)

## 2. Adding more data

At some point we'll need to add data (new rows or columns) to our `DataFrame`. How can we do this?

###  2.1 Adding rows

Let's add a row for Jodrell Bank radio observatory in Cheshire, UK. You'll need to set up a dictionary (`dict`) object where the keys are the column identifiers and the values are the actual row data points.

We can then use the **`append()`** method to add a new row to our `DataFrame`. Note that **`append()`** doesn't actually modify the existing `DataFrame`. Instead it produces a whole **_new_** `DataFrame` with the new data appended. We can assign to a new variable name to this new `DataFrame`. 

Note also that we have to set the parameter `ignore_index=True` when calling **`append()`**.

> A word of warning! The **`append()`** method creates a completely new `DataFrame`, which can use considerable memory for large data sets. There are other, more economical (but often slower) ways to modify `DataFrame`s. 


In [ ]:
# Construct a Python dict containing the new data
new_row = {'Observatory':'Jodrell Bank', 'short':'JB', 
           'lat':53.23625, 'long':-2.307139, 'altitude':77}

# Create a new DataFrame with the new row appended
df_mod = df.append(new_row, ignore_index=True)

# Display the new DataFrame with the row added.
df_mod

### 2.2 Columns

How about adding a column? You can add a column with a simple, default value like this:

```python
df['new_column_name'] = 42
```
As a more useful example, let's add a column, named 'hemisphere', that indicates if the observatory is 'Southern' or 'Northern'. This is a little tricky but can be done using NumPy's **`where()`** function to generate an array of approriate 'Southern' or 'Northern' strings.

We'll pass three arguments to **`where()`**:
1. An `Series` of Boolean values generated by comparing the `lat` column elements of our `DataFrame` with a threshold value of zero - `df_mod['lat'] < 0`
2. A string value to use if `lat` is less than zero - `'Southern'`.
2. A string value to use if `lat` is greater than zero - `'Northern'`.


In [ ]:
df_mod['hemisphere'] = np.where(df_mod['lat'] < 0, 'Southern', 'Northern')
df_mod


## 3. Writing the modified data back to disk

Of course, you can now write this modified data back to a file. _Pandas_ makes this very straightforward. Here's an example using the **`to_csv()`** method.

> The `index=False` argument just ensures that you don't get a column with the row numbers (0,1,2,3,...) in the output file. 


In [ ]:
df_mod.to_csv('observatories-MODIFIED.csv',index=False)


This command creates a new file _and_ writes the data to it. If the file already exists, it will overwrite any pre-existing data in it. You may just want to append the data though, and this can be done in a similar way to how you would do it in 'ordinary' Python - just add a `mode='a'` argument.

```python
df_mod.to_csv('observatories-MODIFIED.csv',index=False, mode='a')
```



# 4. Accessing Sections of your data

How can we use _Pandas_ to get at sub-sets of our data?


### 4.1 Columns

You can access a single column by using the name of the column between square brackets (`[]`). The next cell contains an example.

In [ ]:
df_mod['Observatory']

Note that you can combine this with **`head()`** and **`tail()`** e.g. **`df['Observatory'].head())`**


At the bottom of the output listing, it shows this the returned column has the data type (`dtype`) `object`. In this case, the object is in fact a _Pandas_ `Series`. You could convert this to a NumPy array (**`np.array(df_mod['Observatory'])`** or even a normal Python list (**`list(df_mod['Observatory'])`**).


### EXERCISE 4.1

Display the last 3 rows of the `'short'` column.


In [ ]:
# Write your code here...
df_mod['short'].tail(3)

In [ ]:
df_mod['short'].tail(3)

### 4.2 Rows

Selecting rows is a little less intuitive. In this section, we'll look at using **`.iloc`** to select rows based on their positional indices and _logic-based_ selections to select rows based on Boolean criteria involving their data. Then, later, after talking about indexing we'll show a further method, based on a modified index.

#### 4.2.1 `iloc[]`

First, let's examine the use of **`.iloc[]`**. This function allows rows to be selected based on their _position_ in the `DataFrame`. Its syntax is similar to using the way that slices can be extracted from _NumPy_ arrays e.g. 

```python
numpy_slice = numpy_array[startRow-1:endRow]  
```

The equivalent syntax, using **`.iloc[]`** on a _Pandas_ `DataFrame` would be:

```python
df_slice = df.iloc[startRow-1:endRow]  
```

In fact, just like for NumPy `arrays`, **`.iloc[]`** can be used to extract 2D blocks from `DataFrame`s based on _row_ and _column_ position ranges e.g.
```python
df_slice_2d = df.iloc[startRow-1:endRow, startCol-1:endCol] 
```

Let's look at an example that extracts rows 3 to 5 (`'Green Bank Telescope'` to `'PIRATE'`, indexed as 2,3 and 4 - Remember indices start at 0) from our `DataFrame`.


In [ ]:
df_mod.iloc[2:5,:]


Extending the slicing approach we can restrict the number of columns at the same time:
    

In [ ]:
df_mod.iloc[2:5,1:4]

### 4.3 Logic based selections using `loc[]`

Now for the logic based selection. For this we'll be using **`loc[]`** (without the `i`).

For example, we might just want to use the Southern observatories:

In [ ]:
df_mod.loc[df_mod['lat'] < 0]  # This selects observatories with a latitude less than zero
# Note: In this case df_mod[df_mod['lat'] < 0] (without "loc") will also work just fine.

# Alternatively (and perhaps better) we can search directly on the 'hemisphere' field:
df_mod.loc[df_mod['hemisphere'] == 'Southern']


Or we might want ones that are at an altitude of over 4000 metres:


In [ ]:
df_mod.loc[(df_mod['altitude'] > 4000)]

### EXERCISE 4.3

How would you get all the southern observatories over 4000m high? 

_Hint: You can't use `and` to combine Boolean operations involving `array`s or `Series`. Instead, you'll need to use the `&` operator._


In [ ]:
df_mod.loc[(df_mod['altitude']>4000) & (df_mod['hemisphere']=='Southern')]


In [ ]:
df_mod.loc[(df_mod['altitude'] > 4000) & (df_mod['hemisphere'] == 'Southern')]

 ## 5. Indexing

At the moment, the row index is pretty much like any other list or array - starting at 0 and incrementing. But we can modify this by indexing on another column. Let's say we want to be able to get easy acces to data based on the `'short'` column. 

For this we can use the **`.set_index()`** function. (You can reindex 'inplace' by passing the parameter **`inplace=True`**)

So, using our example and indexing on `'short'` we get:


In [ ]:
df_mod

In [ ]:
# Specify drop=False, so that 'short' is retained as a regular column as 
# well as becoming the index
df_mod.set_index('short', inplace = True, drop =False)
df_mod

**Note:** The `drop=False` argument is important. Without it, the `'short'` column would be removed from the re-indexed `DataFrame`. If we re-indexed the `DataFrame` again, then the data in the `'short'` column would be permanently discarded!

### 5.1. `loc[]` revisited
Now we have a `DataFrame` with an index that is based on the short observatory names, we can demonstrate another way to use **`loc[]`** to select data based on index values.

#### 5.1.1. Selecting whole rows
Let's select all columns (`:`) from the row that has `'ARC'` as its index. 

In [ ]:
row_series = df_mod.loc['ARC',:]
row_series


We can see that **`.loc[]`** has returned another _Pandas_ `Series` containing the row data and indexed by the original `DataFrame` column names. However, as before, you can easily turn it into a `list` or _NumPy_ `array` by using **`df.to_list()`** or **`df.to_numpy()`**. The `Series` object also exposes the individual row data as attributes that can be accessed using the `.` operator. For example:


In [ ]:
print('The longitude of', row_series.Observatory, 'is', row_series.long, 'degrees')
print('It is a', row_series.hemisphere, 'observatory')
numpy_row=row_series.to_numpy()
print('The NumPy array is', numpy_row)

#### 5.1.2. Selecting single values

And, of course, you can get a single value from the `DataFrame` based on its index values in both dimensions.

In [ ]:
df_mod.loc['ARC','altitude']

### EXERCISE 5.1

What is the name of the highest observatory and what are its coordinates (give latitude coordinates in degrees North or South and longitude coordinates in degrees West or East).

_Hints:_ 

1. Find the highest altitude (use the np.max() function on a row).
2. Use this to search for and retrieve the row.
3. Retreive the `lat` and `long`.
4. Do some string formatting stuff.
5. Print it out.


In [ ]:
df_mod

In [ ]:
maxi = np.max(df_mod['altitude'])
h = df_mod.loc[df_mod['altitude']==maxi]
n = h.Observatory [0]
lat = float(h.lat [0])
long = float(h.long[0])

if lat<0:
    latval = str(abs(lat))+'-S'
else:
    latval = str(abs(lat))+'-N'
if long<0:
    lonval = str(abs(long))+'-W'
else:
    lonval = str(abs(long))+'-`e'


print ('The highest is', n, 'the coordinates are:', latval, 'and', lonval)

# Write your code here...

In [ ]:
# Get highest observatory
alt_max = np.max(df_mod['altitude'])
# Which row is this in?
highest_row = df_mod.loc[df_mod['altitude'] == alt_max]
# use the attribute names. In each case a Series is returned with only a single item
# hence we use the [0] to get this.
lat = (float(highest_row.lat[0]))
# and longitude
lon = (float(highest_row.long[0]))
# and the name
nam = highest_row.Observatory[0]
# Construct the latitude string - check if <0, get the string value of
# the absolute value and add '-S' etc. accordingly
if lat<0:
    latval = str(abs(lat))+'-S'
else:
    latval = str(abs(lat))+'-N'
if lon<0:
    lonval = str(abs(lon))+'-W'
else:
    lonval = str(abs(lon))+'-`e'

print('The highest observatory is', nam,'at', lonval, ' ', latval)


## 6. Filtering data (or getting rid of stuff we don't want!)

What if we don't want everything in the `DataFrame`? It is possible to get rid of ("Drop" in Pandas parlance) specific data?

### 6.1 Rows

Use the **`.drop()`** function with a row name (if indexed) or numeric index if not. If it is indexed and you want to use a numeric index, use the **`df.index()`** function. We also need to specify which axis the index to be dropped refers to. Use **`axis=0`** to specify that the index refers to a row and **`axis=1`** to specify that it refers to a column.

Let's remove the `'ERT'` row from our `DataFrame`.


In [ ]:
df_drop = df_mod.drop('ERT',axis=0)
df_drop

How about dropping the 10th row (`'ALMA'`)? We use the **`index[]`** function, remembering that the positional indices start at zero.

In [ ]:
df_drop_num = df_drop.drop(df_drop.index[9], axis=0)
df_drop_num

### 6.2 Columns

To remove columns, we can use almost identical syntax. We just need to specify **`axis=1`** instead of **`axis=0`**.
    
For example, let's discard the `'altitude'` column using its name.

In [ ]:
df_drop_col = df_drop_num.drop('altitude', axis=1)
df_drop_col


## 7. Finally - Getting more control when reading data from a file.

Often we only want to read a subset of the data in a file. _Pandas_ makes it easy to achieve this.

We'll experiment using a CSV file (`Measurements_M13_Lum_00.xls`) with the following column structure (This is an actual data file from a PIRATE observing run):

![CSV file column headers](pirateHeaders.png)

Obviously, some column widths have been collapsed for clarity, but the thing to note is that there are 19 columns headed `'rel_flux_TN'` (with `N` between 2 and 20) and a corresponding set of 19 columns headed `'rel_flux_err_TN'`. Now, what if we're only interested in the first set of data columns - can we restrict which data we read?

The simplest way to do this is to specify the column indices we want to read in and this can be done by using the `usecols` argument and giving it a list of column indices. Note, this is a TAB-delimited file, so we also specify `sep='\t'`.

In [ ]:
pirate_data = pd.read_csv('Measurements_M13_Lum_00.xls', sep = '\t', \
                          usecols = [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])

pirate_data


But, what if we're going to be using data which has variable numbers of the data columns? Well, we can also specify a function as the `usecols` argument, which should determine whether to include the column based on its _name_. For simplicity, we'll use a _Python_ `lambda` function to do this.

In this case, our function looks like:
```python
lambda s: s[0:10]=='rel_flux_T'`
```

This returns `True` (resulting in the colum's inclusion) if the first 10 letters in the column name (which is passed to the function) match `rel_flux_T`


In [ ]:
pirate_data = pd.read_csv('Measurements_M13_Lum_00.xls', sep='\t', usecols = lambda s: s[0:10] == 'rel_flux_T')

pirate_data


The result is the same, but the technique is much more flexible.



# Roundup

So that is just a basic rundown of _Pandas_. It has many more features which are not touched on here, but these should be enough to get you started.


## Just for fun - plotting data

See if you can work out how it's done!


In [ ]:
sorted_data = df_mod.sort_values(by = 'altitude')['altitude']
sorted_data.plot(kind = 'bar', color = 'b')
plt.title('Observatories ranked by altitude')
plt.ylabel('Altitude (m)')
plt.xlabel('Observatory short name')